### Preprocessing

In [1]:
import os
os.chdir("..")

In [3]:
import pandas as pd
import numpy as np
import pickle

from src.data import insert_ids_in_folder, rename_logos, expand_viewbox_in_folder, get_svg_meta_data 
from src.preprocessing import decompose_logos_in_folder, get_path_relevance, apply_embedding_model_to_svgs, encode_svg, decode_z
from src.features import get_style_attributes_folder
from src.animations import create_animated_svg, create_random_animations

#### 1 Rename and insert IDs in SVGs
Logos are renamed and an ID called animation_id is inserted for every path in a SVG.

In [8]:
expand_viewbox_in_folder('data/initial_svgs', 50, "data/svgs_expanded")

In [3]:
rename_logos("data/truncated_svgs_new", "data/truncated_svgs_new_renamed")

In [4]:
insert_ids_in_folder("data/svgs_expanded_renamed", "data/svgs")

#### 2 Decompose SVGs
Decompose SVGs by saving every path in a seperate SVG file. Decomposed SVGs are save to data/decomposed_svgs.

In [4]:
decompose_logos_in_folder("data/svgs")

#### 4 Get SVGs Metadata

In [3]:
pd.set_option('display.max_rows', None)

In [3]:
meta_data = get_svg_meta_data("data/svgs")
meta_data

In [5]:
meta_data.to_csv("data/meta_data/svgs_meta.csv", index=False)

#### 5 Get Path Embeddings
Apply hierarchical generative network to produce latent vector z

In [4]:
apply_embedding_model_to_svgs(data_folder="data/decomposed_svgs", split_paths=True, save=True)

100%|████████████████████████████████████████████████████████████████████████████| 11166/11166 [09:53<00:00, 18.83it/s]


,filename,animation_id,0,1,2,3,4,5,6,7,...,246,247,248,249,250,251,252,253,254,255
0,logo_0,1,0.851117,-1.775123,0.649689,-0.688600,0.216071,0.135211,-1.748761,-1.347670,...,-1.616431,-1.572003,0.242460,0.430259,0.079752,-1.039526,-0.696104,0.090277,-0.228757,0.144372
1,logo_0,0,0.763518,-0.982797,-0.446681,1.089468,-0.070563,0.710206,-0.491675,-1.631172,...,0.339061,0.022934,0.195161,-0.046488,-0.492103,-0.605836,-1.282879,0.613195,0.297194,-0.172312
2,logo_0,2,0.328628,-0.612160,1.592824,-0.422598,-0.765523,1.140459,0.145881,0.240300,...,-0.254159,-0.790789,-0.057158,1.594653,0.395605,-1.048015,0.454987,-0.439861,1.430237,0.192277
3,logo_0,3,0.291136,-0.928242,0.265542,-0.261439,-0.386160,1.256256,-0.414706,-1.206105,...,-0.446020,-1.369758,0.356421,1.456656,0.468766,-1.077724,-0.548627,-0.300660,0.632805,-0.136473
4,logo_0,4,0.504446,-0.543099,0.915062,1.293575,-0.849605,1.120387,-0.637641,-1.337280,...,0.162753,0.206993,-0.201259,-1.087391,-0.597388,-0.992079,-0.851486,-0.225463,-0.549269,0.088637
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10804,logo_9,3,0.341127,-0.844122,0.851799,-0.352970,-0.859344,0.568410,-1.287803,-0.416381,...,-1.519855,-4.074135,-0.355853,1.073493,1.398292,-1.681473,-1.260963,-0.233300,-0.015946,0.974122
10805,logo_9,6,0.424217,-1.353261,1.238835,-0.837542,-0.345199,-0.974526,-1.683693,-0.507383,...,-3.137928,-3.240320,0.277688,-0.590543,0.265793,-1.681521,-0.566898,0.476020,-0.685451,1.096683
10806,logo_9,7,1.531518,-1.612310,0.801471,1.137218,0.327335,1.749122,-0.426959,-0.576917,...,1.142574,-1.827454,-0.635960,2.076138,0.755637,-0.566260,-0.688139,-0.606805,0.903784,-0.709454
10807,logo_9,5,0.621977,-2.617885,1.425599,-0.207073,0.257930,-0.073098,-2.433338,-0.637954,...,-2.948898,-3.191816,0.459644,0.794539,0.686447,-1.263997,-0.288715,0.684191,-0.894395,0.695853


In [10]:
with open('data/embeddings/hierarchical_ordered_decomposed_svgs_embedding.pkl', 'rb') as f:
    path_embedding = pickle.load(f)

#### 7 Get Style Attributes
Get local as well as global style attributes of SVGs.

In [7]:
df_style_attributes = get_style_attributes_folder("data/svgs")
df_style_attributes